In [ ]:
# import packages
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk  
import tensorflow as tf
!pip install transformers

import torch

# BERT and T5 are easily imported from Hugging Face's transformer library (https://huggingface.co/transformers/model_doc/bert.html#tfbertmodel
from transformers import BertTokenizer, TFBertModel, T5Tokenizer,  MT5Tokenizer, TFT5ForConditionalGeneration
from torch.utils.data import TensorDataset, random_split

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.utils import class_weight 
from sklearn.model_selection import GridSearchCV



# Keras functional API
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GlobalMaxPooling1D
from keras.layers import SpatialDropout1D
from keras.layers import Input

import math

!pip install pyspellchecker  
from spellchecker import SpellChecker
  
import random
from random import randint
import more_itertools
from nltk import tokenize
nltk.download('punkt')

tf.test.gpu_device_name()



     |████████████████████████████████| 2.6 MB 12.4 MB/s 
     |████████████████████████████████| 636 kB 66.7 MB/s 
     |████████████████████████████████| 3.3 MB 63.0 MB/s 
     |████████████████████████████████| 895 kB 48.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


     |████████████████████████████████| 2.7 MB 13.9 MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


'/device:GPU:0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
# import transformers
#!pip install -q transformers

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test_set.tsv",sep='\t', encoding='ISO-8859-1')
dev = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/valid_set.tsv",sep='\t', encoding='ISO-8859-1')
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/training_set_rel3.tsv",sep='\t', encoding='ISO-8859-1')

In [ ]:
def clean_anonymization(essay):
  res=[]
  for i in essay.split():
    if i.startswith("@"):
      continue
    else:
      res.append(i)
  return ' '.join(res)

train['essay']=train['essay'].apply(lambda x:clean_anonymization(x))
# also remove from dev and test
dev['essay']=dev['essay'].apply(lambda x:clean_anonymization(x))
test['essay']=test['essay'].apply(lambda x:clean_anonymization(x))

In [ ]:
def first_stage_model(hidden_size = 100, train_layers = -1, learning_rate= 1e-5, dropout_value=0.3):

    optimizer=tf.keras.optimizers.Adam(learning_rate)   

    """
    Build a simple classification model with BERT. Let's keep it simple and don't add dropout, layer norms, etc.
    """

    input_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                  'token_type_ids': token_type_ids,
                  'attention_mask': attention_mask}
    
    #restrict training to the train_layers outer transformer layers
    if not train_layers == -1:

            retrain_layers = []

            for retrain_layer_number in range(train_layers):

                layer_code = '_' + str(11 - retrain_layer_number)
                retrain_layers.append(layer_code)

            for w in bert_model.weights:
                if not any([x in w.name for x in retrain_layers]):
                    w._trainable = False


    bert_out = bert_model(bert_inputs)

    classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(bert_out[0])

    hidden = tf.keras.layers.Dense(hidden_size, name='hidden_layer')(classification_token)

    dropout = tf.keras.layers.Dropout(dropout_value)(hidden)

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(dropout)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], 
                                          outputs=[classification])
    
    classification_model.compile(optimizer=optimizer,
                            loss="mse",
                            metrics=["mae"])


    return classification_model

In [ ]:
# Create a dataframe for adding prompt of each essay set to get the prompt-relevant score
prompt = pd.DataFrame({"essay_set":[1,2,3,4,5,6,7,8],
                       "prompt":["More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about faraway places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends. Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you.",
                                "Write a persuasive essay to a newspaper reflecting your vies on censorship in libraries. Do you believe that certain materials, such as books, music, movies, magazines, etc., should be removed from the shelves if they are found offensive? Support your position with convincing arguments from your own experience, observations, and/or reading.",
                                "Write a response that explains how the features of the setting affect the cyclist. In your response, include examples from the essay that support your conclusion.",
                                "Write a response that explains why the author concludes the story with this paragraph. In your response, include details and examples from the story that support your ideas.",
                                "Describe the mood created by the author in the memoir. Support your answer with relevant and specific information from the memoir.",
                                "Based on the excerpt, describe the obstacles the builders of the Empire State Building faced in attempting to allow dirigibles to dock there. Support your answer with relevant and specific information from the excerpt.",
                                "Write about patience. Being patient means that you are understanding and tolerant. A patient person experience difficulties without complaining.Do only one of the following: write a story about a time when you were patient OR write a story about a time when someone you know was patient OR write a story in your own way about patience.",
                                "We all understand the benefits of laughter. For example, someone once said, “Laughter is the shortest distance between two people.” Many other people believe that laughter is an important part of any relationship. Tell a true story in which laughter was one element or part."]})
prompt

,essay_set,prompt
0,1,"More and more people use computers, but not ev..."
1,2,Write a persuasive essay to a newspaper reflec...
2,3,Write a response that explains how the feature...
3,4,Write a response that explains why the author ...
4,5,Describe the mood created by the author in the...
5,6,"Based on the excerpt, describe the obstacles t..."
6,7,Write about patience. Being patient means that...
7,8,We all understand the benefits of laughter. Fo...


In [ ]:
def normalize(df):
    df['normalized_score'] = df['domain1_score'] / df.groupby('essay_set')['domain1_score'].transform('max')
    return df

def tokenized(df, examples):
    example_iter = random.sample(range(0, len(df)-1), examples)
    input_ids = []
    token_type_ids = []
    attention_mask = []
    example = []
    gold_scores = []
    for iteration in example_iter:
      essay = df['essay'].iloc[iteration]
      encoded_dict = tokenizer(essay, 
                max_length=512,
                padding='max_length',
                truncation = True, 
                return_tensors='tf')
      example.append(iteration)
      input_ids.append(encoded_dict.input_ids)
      token_type_ids.append(encoded_dict.token_type_ids)
      attention_mask.append(encoded_dict.attention_mask)
      gold_scores.append(df['normalized_score'].iloc[iteration])

    return {'gold_scores': gold_scores,'input_ids': tf.concat([z for z in input_ids], axis=0), 'token_type_ids': tf.concat([z for z in token_type_ids], axis=0), 'attention_mask': tf.concat([z for z in attention_mask], axis=0), 'example': example}


def tokenized_test(df, examples):
    example_iter = random.sample(range(0, len(df)), examples)
    input_ids = []
    token_type_ids = []
    attention_mask = []
    example = []
    for iteration in example_iter:
      essay = df['essay'].iloc[iteration]
      encoded_dict = tokenizer(essay, 
                max_length=512,
                padding='max_length',
                truncation = True, 
                return_tensors='tf')

      example.append(df['essay_id'].iloc[iteration])
      input_ids.append(encoded_dict.input_ids)
      token_type_ids.append(encoded_dict.token_type_ids)
      attention_mask.append(encoded_dict.attention_mask)
    
    return {'input_ids': tf.concat([z for z in input_ids], axis=0), 'token_type_ids': tf.concat([z for z in token_type_ids], axis=0), 'attention_mask': tf.concat([z for z in attention_mask], axis=0), 'example': example}


In [ ]:
train = normalize(train)
y = train['normalized_score']

X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.33, random_state=0)
enc_train = tokenized(X_train, len(X_train) - 1)
enc_val = tokenized(X_test, len(X_test) - 1)
class_weights = class_weight.compute_class_weight('balanced',pd.Series(enc_train['gold_scores']).value_counts().index,pd.Series(enc_train['gold_scores']))
class_weights = dict(enumerate(class_weights))
fsm = first_stage_model()
fsm.fit([enc_train['input_ids'], enc_train['token_type_ids'], enc_train['attention_mask']], pd.Series(enc_train['gold_scores']), validation_data = ([enc_val['input_ids'], enc_val['token_type_ids'], enc_val['attention_mask']], pd.Series(enc_val['gold_scores'])) , epochs=2, batch_size=10, class_weight=class_weights, shuffle=True)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
combined_adv = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/combined5.csv")
y = combined_adv['normalized_score']
combined_adv['sent'] = combined_adv['essay'].apply(lambda x: tokenize.sent_tokenize(x))
enc_test = tokenized_test(combined_adv, len(combined_adv))

a = fsm.predict([enc_test['input_ids'], enc_test['token_type_ids'], enc_test['attention_mask']])
res = pd.DataFrame(a)
res.index = enc_test['example'] # its important for comparison
res.columns = ["semantic_prediction"]
res.to_csv("semantic_combined_adv_results3.csv")

In [ ]:
res.sort_index()

,prediction
1,0.691455
2,0.731046
3,0.615243
4,0.791722
5,0.762809
...,...
21626,0.584657
21628,0.518016
21629,0.606764
21630,0.623187


In [ ]:
train.essay_id

0            1
1            2
2            3
3            4
4            5
         ...  
12971    21626
12972    21628
12973    21629
12974    21630
12975    21633
Name: essay_id, Length: 12976, dtype: int64

#Hyperparameter Tuning


In [ ]:
train = normalize(train)
y = train['normalized_score']

X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.33, random_state=0)
enc_train = tokenized(X_train, 1500)
enc_val = tokenized(X_test, 1500)
class_weights = class_weight.compute_class_weight('balanced',pd.Series(enc_train['gold_scores']).value_counts().index,pd.Series(enc_train['gold_scores']))
class_weights = dict(enumerate(class_weights))

dropout_values = [0.1,0.3, 0.5]
hidden_sizes = [100, 300, 500]
batch_sizes = [10]
learning_rates = [1e-5, 3e-5]
histories = {}
i = 0
for dropout_value in dropout_values:
  for hidden_size in hidden_sizes:
    for batch_size in batch_sizes:
      for learning_rate in learning_rates:
        print('\n' + 'iteration ' + str(i) + ' dropout_value: ' + str(dropout_value) + ' hidden_size: ' + str(hidden_size) + ' batch_size: ' + str(batch_size) + ' learning_rate ' + str(learning_rate) + "\n")
        fsm = first_stage_model(hidden_size=hidden_size, dropout_value=dropout_value, learning_rate=learning_rate)
        history = fsm.fit([enc_train['input_ids'], enc_train['token_type_ids'], enc_train['attention_mask']], pd.Series(enc_train['gold_scores']), validation_data = ([enc_val['input_ids'], enc_val['token_type_ids'], enc_val['attention_mask']], pd.Series(enc_val['gold_scores'])) , epochs=2, batch_size=batch_size, class_weight=class_weights, shuffle=True, verbose=2)
        histories[(dropout_value, hidden_size, batch_size, learning_rate)] = history.history['val_loss']
        i += 1


iteration 0 dropout_value: 0.1 hidden_size: 100 batch_size: 10 learning_rate 1e-05

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not y

In [ ]:
histories

{(0.1, 100, 10, 1e-05): [0.02456650510430336, 0.02800252102315426],
 (0.1, 100, 10, 3e-05): [0.021624188870191574, 0.021066058427095413],
 (0.1, 300, 10, 1e-05): [0.024605130776762962, 0.020202169194817543],
 (0.1, 300, 10, 3e-05): [0.023734863847494125, 0.02304534986615181],
 (0.1, 500, 10, 1e-05): [0.019976699724793434, 0.020669274032115936],
 (0.1, 500, 10, 3e-05): [0.022997837513685226, 0.02453004941344261],
 (0.3, 100, 10, 1e-05): [0.019780432805418968, 0.018852833658456802],
 (0.3, 100, 10, 3e-05): [0.022271106019616127, 0.02014048583805561],
 (0.3, 300, 10, 1e-05): [0.0191819965839386, 0.023007797077298164],
 (0.3, 300, 10, 3e-05): [0.022931409999728203, 0.02006475254893303],
 (0.3, 500, 10, 1e-05): [0.019553102552890778, 0.019102487713098526],
 (0.3, 500, 10, 3e-05): [0.01931898668408394, 0.02022736333310604],
 (0.5, 100, 10, 1e-05): [0.019699934870004654, 0.020651793107390404],
 (0.5, 100, 10, 3e-05): [0.02075960859656334, 0.020776918157935143],
 (0.5, 300, 10, 1e-05): [0.0199

In [ ]:
h = pd.DataFrame.from_dict(histories)
h.to_csv('tuning_semantic.csv')